In [1]:
import pandas as pd
import numpy as np
import re
import math


# ----------------------------------------------------------------------------------------------------
#         !!! A changer pour prendre le bon path d'entre et de sortie du fichier csv !!!
# ----------------------------------------------------------------------------------------------------
path_entre = r'../ressources/babyfoot_dataset.csv'

path_sortie = r'./babyfoot_dataset_clear.csv'

# ----------------------------------------------------------------------------------------------------

df = pd.read_csv(path_entre, sep=",")          # chargement de la base de donnée

/var/folders/f4/5gyrd33x1pz_mbph3rmmk_1w0000gn/T/ipykernel_6510/3901812418.py:16: DtypeWarning: Columns (10,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_entre, sep=",")          # chargement de la base de donnée


In [2]:
# -----------------------------------------------------------
#           Nettoyage des données
# -----------------------------------------------------------


df.drop(columns=['mood', 'misc', 'created_at', 'player_name', 'ping_ms', 'player_comment', 'notes', 'duplicate_flag', 'winner'], inplace=True)


df["game_date"] = pd.to_datetime(df["game_date"], errors="coerce", dayfirst=True)                       # nettoyage des dates
df["game_date"] = df["game_date"].dt.strftime("%d/%m/%Y")                                               # formats type jour/mois/année


df["location"] = df["location"].replace(['Ynov Tls', 'Ynov - Bâtiment A'], 'Ynov Toulouse')             # Normalisation de location
df["location"] = df["location"].replace(['Cafeteria (1st floor)', 'Campus - Cafeteria',], 'Cafeteria')
df["location"] = df["location"].replace('Student House�', 'Student House')

df["table_condition"] = df["table_condition"].replace('good�', 'good')                              # Normalisation de table_condition
df["table_condition"] = df["table_condition"].replace('scratched�', 'scratched')
df["table_condition"] = df["table_condition"].replace('missing screw�', 'missing screw')

df["season"] = df["season"].replace(['Season 24-25', '2025 Season', 's24/25'], '2024/2025')             # Normalisation de season

df["player_age"] = df["player_age"].replace(['twenty', 'twenty�'], '20')                            # Normalisation de player_age
df["player_age"] = df["player_age"].replace(['21 yrs', 'twenty�'], '21')


df["player_role"] = df["player_role"].replace(['defence', 'defense', 'Defense'], 'def')          # Normalisation de player_role
df["player_role"] = df["player_role"].replace(['ATTACK', 'attck', 'attack', 'attck�'], 'atk')

df["player_goals"] = df["player_goals"].replace('0�', '0')                                          # Normalisation de player_goals
df["player_goals"] = df["player_goals"].replace('one', '1')
df["player_goals"] = df["player_goals"].replace(['two', '2�'], '2')
df["player_goals"] = df["player_goals"].replace('three', '3')
df["player_goals"] = df["player_goals"].replace('ten', '10')

df["player_assists"] = df["player_assists"].replace('0�', '0')                                         # Normalisation de player_assists
df["player_assists"] = df["player_assists"].replace('5�', '5')


df["player_assists"] = df["player_assists"].apply(lambda x: int(x) if str(x).isdigit() else 0)              # transforme les nombre sous forme de text en int
df["player_own_goals"] = df["player_own_goals"].apply(lambda x: int(x) if str(x).isdigit() else 0)
df["player_goals"] = df["player_goals"].apply(lambda x: int(x) if str(x).isdigit() else 0)
df["player_age"] = df["player_age"].apply(lambda x: int(x) if str(x).isdigit() else 0)





conversion_map = {                                                                # Normalisation de rating_raw
    'one': 1, '1': 1,
    'two': 2, '2': 2,
    'three': 3, '3': 3,
    'four': 4, '4': 4,
    'five': 5, '5': 5,
    '👍': 5, '🙂': 4, '😡': 1,
}

def convert_rating(value):
    value = str(value).lower().strip()
    if '⭐' in value:
        return value.count('⭐')
    return conversion_map.get(value, np.nan)

df['rating_raw'] = df['rating_raw'].apply(convert_rating).astype('Int64')






df['attendance_count'] = (                                                                   # Normalisation de attendance_count
    df['attendance_count']
    .astype(str)
    .str.extract('(\d+)')
    .astype(float)
    .astype('Int64')
)







df['team_color'] = df['team_color'].replace({                                                                 # Normalisation de team_color
    'Red': 'red', 'red': 'red', 'R': 'red', '🔴': 'red', 'Red�': 'red',
    'Blue': 'blue', 'blue': 'blue', 'B': 'blue', '🔵': 'blue', 'Blue�': 'blue'
})
 





print(df['is_substitute'].unique())                                                                         # Normalisation de is_substiture
df['is_substitute'] = df['is_substitute'].replace({
    'yes': 'yes', 'Yes': 'yes', 'YES': 'yes',
    'no': 'no', 'No': 'no', 'NO': 'no', 'no�': 'no',
    'maybe': 'maybe', 'Maybe': 'maybe', 'MAYBE': 'maybe',
    np.nan: np.nan
})
 





# (The 'convertir_en_secondes' function needs modification for direct 'apply')

def convertir_en_secondes_cellule(element):                                                                   # Normalisation de possession_time
    """
    Adapte la fonction pour traiter un SEUL élément de la cellule de DataFrame
    dans les formats M:S ou M.dmin.
    """
    # Votre logique de conversion, simplifiée pour une seule valeur
    try:
        if isinstance(element, str):
            element = element.strip()
        else:
            # Si ce n'est pas une chaîne (ex: float ou int), on essaie de convertir directement en secondes
            return int(element) * 60 if '.' not in str(element) else int(round(float(element) * 60))


        if ':' in element:
            minutes, secondes = map(int, element.split(':'))
            total_secondes = (minutes * 60) + secondes
            return total_secondes

        elif 'min' in element:
            valeur_str = element.replace('min', '')
            minutes_decimales = float(valeur_str)

            # Conversion de la partie décimale
            total_secondes = int(round(minutes_decimales * 60))

            return total_secondes

        else:
            # Si c'est juste un nombre entier sans unité (on suppose que c'est déjà en secondes)
            return int(element)

    except (ValueError, TypeError):
        # Retourne NaN pour les valeurs non convertibles
        return pd.NA

df['possession_time'] = df['possession_time'].apply(convertir_en_secondes_cellule)






df = df[~df.duplicated(subset=["game_id", "player_id"])]                                        # supprime les dublication des combinaise game_id & player_id




def convert_to_seconds(value):                                                              # Normalisation de game_duration
    """Convertit un format comme '12min', '00:08:11', '5.45' (MM.SS) en secondes"""
    value = str(value).strip()
    
    value = ''.join(c for c in value if c.isalnum() or c in ':.-')

    if ':' in value:
        parts = value.split(':')
        if len(parts) == 2:
            return int(parts[0]) * 60 + int(parts[1])
        elif len(parts) == 3:
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])

    elif 'min' in value:
        try:
            return int(value.replace('min', '')) * 60
        except:
            return None

    elif '.' in value:
        try:
            parts = value.split('.')
            minutes = int(parts[0])
            seconds = int(parts[1])
            
            if seconds >= 60:
                minutes += seconds // 60
                seconds = seconds % 60
            
            return minutes * 60 + seconds
        except:
            return None

    else:
        try:
            return float(value)
        except:
            return None

df['game_duration'] = df['game_duration'].apply(convert_to_seconds)






def clean_strange_chars(df):                                                           # enleve les symbole à la fin
    for col in df.columns:
        if df[col].dtype == "object":  # uniquement les colonnes texte
            df[col] = df[col].astype(str).str.replace(r'�+$', '', regex=True).str.strip()
    return df

clean_strange_chars(df)




RE_TWO = re.compile(r'(\d{1,3})\s*[-:/]\s*(\d{1,3})')  # Match "X - Y"                                   # Normalisation de final_score_red & final score_blue
RE_NUM = re.compile(r'(\d{1,3})')  # Match single number anywhere

def normalize_scores_row_final_v2(row):
    b_raw = row.get('final_score_blue')
    r_raw = row.get('final_score_red')

    b_s = '' if pd.isna(b_raw) else str(b_raw)
    r_s = '' if pd.isna(r_raw) else str(r_raw)

    # 1️⃣ RED contient "X - Y" → priorité absolue
    m = RE_TWO.search(r_s)
    if m:
        red = int(m.group(1))
        blue = int(m.group(2))
        return pd.Series({'final_score_blue': blue, 'final_score_red': red})

    # 2️⃣ RED contient un nombre, BLUE contient un nombre → prise directe
    m_r = RE_NUM.search(r_s)
    m_b = RE_NUM.search(b_s)
    if m_r and m_b:
        return pd.Series({'final_score_blue': int(m_b.group(1)), 'final_score_red': int(m_r.group(1))})

    # 3️⃣ RED = un nombre mais BLUE vide
    if m_r and not m_b:
        return pd.Series({'final_score_blue': np.nan, 'final_score_red': int(m_r.group(1))})

    # 4️⃣ BLUE = un nombre mais RED vide
    if m_b and not m_r:
        return pd.Series({'final_score_blue': int(m_b.group(1)), 'final_score_red': np.nan})

    # 5️⃣ Rien de lisible
    return pd.Series({'final_score_blue': np.nan, 'final_score_red': np.nan})

fixed = df.apply(normalize_scores_row_final_v2, axis=1)
df[['final_score_blue', 'final_score_red']] = fixed




df["winner"] = df.apply(                                                                         # reecriture de winner par rapport au score_final_red & score_final_blue
    lambda row: "blue" if row["final_score_blue"] > row["final_score_red"] 
    else "red" if row["final_score_red"] > row["final_score_blue"] 
    else "equal",
    axis=1
)

# -----------------------------------------------------------
df.to_csv(path_sortie,index=False)   # export du fichier csv
# -----------------------------------------------------------

['yes' 'no' 'maybe' nan 'no�']


KeyboardInterrupt: 

In [ ]:
# -----------------------------------------------------------
#       Défi Data Science version python (il existe aussi la version sql)
# -----------------------------------------------------------


top_10_goal = (
    df.groupby("player_id")['player_goals']
      .sum()
      .sort_values(ascending=False)
      .head(10)
      .reset_index()
)

top_5_save = (
    df.groupby("player_id")["player_saves"]
      .sum()
      .sort_values(ascending=False)
      .head(5)
      .reset_index()
)

best_color = df["winner"].value_counts(normalize=True) * 100

print(top_10_goal)
print(top_5_save)
print(best_color)
# -----------------------------------------------------------

  player_id  player_goals
0     P0514           446
1     P0416           425
2     P0234           421
3     P0217           413
4     P0237           411
5     P0191           408
6     P0245           407
7     P0177           407
8     P0405           407
9     P0354           405
  player_id  player_saves
0     P0405           846
1     P0416           834
2     P0241           817
3     P0719           817
4     P0062           808
red      45.864
blue     45.144
equal     8.992
Name: winner, dtype: float64
